In [1]:
function calculate_columns(node)
    node_pool=Array{Array{Array{Int,1},1},1}(undef,nv(new_inst.graph))

    for l in 1:nv(new_inst.graph)
        node_pool[l]=Array{Array{Int,1},1}()
        push!(node_pool[l],vcat(10000,ones(Int,nv(new_inst.graph))))
        for k in 2:size(column_pool[l],1)
            add=true
            
            # There are two conditions under which the column should be eliminated:
            #  1. If the branching constraint x_ji<=0 apply and job i is assigned to machine j
            for (i,j) in tree[node].setzero
                if column_pool[l][k][i+1]==1
                    add=false
                end
            end
            #  2. If the branching constraint x_ji>=1 apply and job i is not assigned to machine j
            for (i,j) in tree[node].setone
                if column_pool[l][k][i+1]==0
                    add=false
                end
            end

            if add
                push!(node_pool[l],column_pool[l][k])
            end
        end
    end

    return node_pool
end

calculate_columns (generic function with 1 method)

In [ ]:
# Transformer la solution α en x
function calculate_xsol(mastersol,node_pool)
    x=Array{Array{Float32},1}(undef,nv(new_inst.graph))
    for i in 1:nv(new_inst.graph)
        x[i]=Array{Float32,1}(undef,nv(new_inst.graph))
    end

    for i in 1:nv(new_inst.graph)
        for j in 1:nv(new_inst.graph)
            x[i][j]=sum(node_pool[i][k][j+1]*mastersol[i][k] for k in 1:size(mastersol[i],1))
        end
    end
    print("x",x)
    print("\n")
    return x
end

In [7]:
function Process_Node(nodeindex)
    println("\e[93m Noeud traité : $nodeindex \e[00m")
    println("Variables mises à 0 sont :", tree[nodeindex].setzero)
    println("Variables mises à 1 sont :", tree[nodeindex].setone)
    global nodestobedeleted = []
    
    # Calcul des colonnes compatibles avec les contraintes de branchements
    node_pool = calculate_columns(nodeindex)
    
    # On prend le premier maitre restreint
    sommets,ylc,mastermodel=node_master(node_pool)
    
    while true
        # Résolution du maitre restreint
        optimize!(mastermodel)
        
        node_infeasible = false
        
        # Stockage de la solution duale 
        π = JuMP.dual.(sommets)
        
        # Stockage de la valeur objective du problème maitre actuel
        value = -JuMP.objective_value(mastermodel)
    
        # Stockage de la solution
        solution=Array{Array{Float32,1},1}(undef,nv(new_inst.graph))
        for l in 1:nv(new_inst.graph)
            solution[l]=Array{Float32,1}(undef,size(node_pool[l],1))
            for j in 1:size(node_pool[l],1)
               solution[l][j]=JuMP.value.(ylc[l])[j]
            end
        end
        
        solflat = collect(Iterators.flatten(solution))
      
        # Vérification si la solution est entière
        if maximum(solflat-floor.(solflat)) <= ϵ
            if value <= UB
                global UB=value
            end
            println("\e[32m Solution admissible avec valeur $value trouvée \e[00m")
            
            for i in 1:length(Queue)-1
                if tree[Queue[i]].lb >= UB
                    push!(nodestobedeleted,i)
                end
            end
        end
 
        # Mise a jour des bornes supérieure et inférieure au noeud actuel
        nodeub = value
        nodelb = sum(π[i] for i in 1:nv(new_inst.graph))
        print("nodelb : ", nodelb," \n")
        minobj = 0
        
        # Résolution des sous-problèmes
        for l in 1:nv(new_inst.graph)
            SPobj,column = subproblem(l,π,nodeindex)

            print("spob : ",SPobj,"\n")
            print("column : ",column,"\n")
            
            if SPobj<Inf 
                # Extraction du motif de la solution
                pattern=findall(x->x!=0,column)
                # Calcul du cout réduit du motif
                reduced_cost = SPobj
                if reduced_cost<=minobj 
                    minobj=reduced_cost
                end

                # Si un motif à cout réduit est trouvé, on l'ajoute aux patterns
                if reduced_cost<=0 # car maximisation
                    # Calcul du cout de la colonne qui sera une contribution à la fonction objective
                    cout_colonne=sum(column[j] for j in 1:size(column,1))
                    # Ajout colonne dans column_pool
                    push!(column_pool[l],vcat(cout_colonne,column))

                    # Ajout colonne dans node_pool
                    push!(node_pool[l],vcat(cout_colonne,column))
                
                    # Ajout de la colonne au modèle
                    # Ajout de la nouvelle variable au vecteur des variables
                    push!(ylc[l],@variable(mastermodel,lower_bound=0))
                    # Mis à jour du nom
                    set_name(ylc[l][end], "ylc_$l,$(size(column_pool,1))")
                    # Donner les coefficients aux variables dans les contraintes liées aux paires
                    for j in 1:nv(new_inst.graph)
                        set_normalized_coefficient(sommets[j], ylc[l][end], column[j])
                    end
                
                    # Mise a jour de la fonction objective
                    set_objective_function(mastermodel, objective_function(mastermodel) + cout_colonne*ylc[l][end])
                
                    println("\e[34m Cycle ajouté pour $l \e[00m")
                end
                # Calcul de la nouvelle borne inférieure
                nodelb += SPobj
            else
                node_infeasible = true
                println("\e[31m Noeud infaisable \e[00m")
                break
            end
        end
        
        # Si le sous-problème n'est pas faisable alors le noeud n'est pas faisable
        if node_infeasible
            return []
        end
      
        # Si la borne inférieure est améliorée, alors on la met à jour
        if nodelb>=tree[nodeindex].lb
            tree[nodeindex].lb=nodelb
            print("nodelb : ",nodelb,"\n")
        end
        
        # Si il y a convergence, on s'arrête
        if 2*abs((nodeub-tree[nodeindex].lb))/abs((nodeub+tree[nodeindex].lb))<ϵ || minobj>=0
            println("La relaxation au noeud est résolue à l'optimalité")
            println("La borne supérieure au noeud est $nodeub")
            println("La borne inférieure au noeud est $(tree[nodeindex].lb)")
    
            # Si la colonne artificielle est utilisée dans la solution finale alors le maitre est infaisable
            for l in 1:nv(new_inst.graph)
                if solution[l][1]>=ϵ
                    node_infeasible = true
                    println("\e[31m Noeud est infaisable \e[00m")
                end
            end 
            if node_infeasible
                return []
            else
                return calculate_xsol(solution,node_pool)
            end
        end
    end
end

LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[7]:86

In [8]:
tab=[[1,2,3],[4,5,6]]
a = vcat(5,[1,2,3,4])
push!(tab,a)
print(tab)

[[1, 2, 3], [4, 5, 6], [5, 1, 2, 3, 4]]